In [ ]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy.stats import zscore
import matplotlib.pyplot as plt

In [ ]:
# file path
raw_df = pd.read_csv('/content/PID_197_cleanData.csv')

## text data

In [ ]:
text_df = raw_df[["Response_ID","PID","Condition","text_Likelihood_to_Accept",\
                  "text_Explanation_Feedback_1","text_Explanation_Feedback_2","Likelihood_to_Accept_1"]]

# combine 3 questions
text_df["combine_text"] = text_df["text_Likelihood_to_Accept"] + text_df["text_Explanation_Feedback_1"] + text_df["text_Explanation_Feedback_2"]

# label accept
def label_Accept(x):
    if x > 70:
        return 1
    else:
        return 0

text_df["label_Likelihood_to_Accept_1"]= text_df["Likelihood_to_Accept_1"].apply(label_Accept)


text_df.head()


/tmp/ipython-input-2834432379.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df["combine_text"] = text_df["text_Likelihood_to_Accept"] + text_df["text_Explanation_Feedback_1"] + text_df["text_Explanation_Feedback_2"]
/tmp/ipython-input-2834432379.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df["label_Likelihood_to_Accept_1"]= text_df["Likelihood_to_Accept_1"].apply(label_Accept)


,Response_ID,PID,Condition,text_Likelihood_to_Accept,text_Explanation_Feedback_1,text_Explanation_Feedback_2,Likelihood_to_Accept_1,combine_text,label_Likelihood_to_Accept_1
0,36127401,1,1,I would naturally accept the first treatment o...,I feel the information provided by the AI doct...,"Certainly, as long as it's not the only option...",98,I would naturally accept the first treatment o...,1
1,36132699,2,2,"the treatment suggestion was sensible, low-ris...","clear and easy to understand information, seem...","yes, but I would not accept it as a replacemen...",95,"the treatment suggestion was sensible, low-ris...",1
2,36147343,4,4,It is a relatively minor condition and the tre...,It was quite brief and concise. I think that i...,Not unless a real doctor was not available or ...,80,It is a relatively minor condition and the tre...,1
3,36149983,5,5,I think the condition in my mind was moderatel...,"It was a lot of information to digest, and som...",Probably not unless it was something very mino...,67,I think the condition in my mind was moderatel...,0
4,36154834,7,7,the AI doctor examined the only place where th...,The information was brief and non-descriptive ...,Yes for minor illnesses that can require a sim...,65,the AI doctor examined the only place where th...,0


In [ ]:
# remove redundant whitespace
text_df['combine_text'] = text_df['combine_text'].str.strip()

texts = text_df['combine_text'].tolist()

/tmp/ipython-input-1030543678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['combine_text'] = text_df['combine_text'].str.strip()


## LLM & RAG
https://huggingface.co/blog/ngxson/make-your-own-rag

https://github.com/langchain-ai/rag-from-scratch

In [ ]:
# Install LlamaIndex
# RAG is a technique that allows an LLM to retrieve relevant information
# from an external knowledge base when generating responses.
# ramework such as LlamaIndex or LangChain.
# build data into a vector database for retrieval.

!pip install llama-index

In [ ]:
!pip install transformers accelerate


In [ ]:
%pip install llama-index-embeddings-huggingface

  Using cached llama_index_embeddings_huggingface-0.6.0-py3-none-any.whl.metadata (458 bytes)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink

In [ ]:
!pip install llama-index llama-index-llms-huggingface

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
# https://huggingface.co/blog/ngxson/make-your-own-ra
#!pip install ollama

In [ ]:
# Loading the dataset
dataset = texts
print(f'Loaded {len(dataset)} entries')

# Implement the vector database
#import ollama
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.readers.base import BaseReader
from llama_index.core.schema import BaseComponent, Document
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding



EMBEDDING_MODEL = 'BAAI/bge-small-en'
LANGUAGE_MODEL = 'meta-llama/Llama-2-7b-chat-hf'

embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)

# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []
def add_chunk_to_database(chunk):
    embedding = embed_model.get_text_embedding(chunk)
    VECTOR_DB.append((chunk, embedding))

# def add_chunk_to_database(chunk):
#   embedding = HuggingFaceEmbedding(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
#   VECTOR_DB.append((chunk, embedding))

for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  print(f'Added chunk {i+1}/{len(dataset)} to the database')



Loaded 132 entries


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Added chunk 1/132 to the database
Added chunk 2/132 to the database
Added chunk 3/132 to the database
Added chunk 4/132 to the database
Added chunk 5/132 to the database
Added chunk 6/132 to the database
Added chunk 7/132 to the database
Added chunk 8/132 to the database
Added chunk 9/132 to the database
Added chunk 10/132 to the database
Added chunk 11/132 to the database
Added chunk 12/132 to the database
Added chunk 13/132 to the database
Added chunk 14/132 to the database
Added chunk 15/132 to the database
Added chunk 16/132 to the database
Added chunk 17/132 to the database
Added chunk 18/132 to the database
Added chunk 19/132 to the database
Added chunk 20/132 to the database
Added chunk 21/132 to the database
Added chunk 22/132 to the database
Added chunk 23/132 to the database
Added chunk 24/132 to the database
Added chunk 25/132 to the database
Added chunk 26/132 to the database
Added chunk 27/132 to the database
Added chunk 28/132 to the database
Added chunk 29/132 to the dat

In [ ]:
# Implement the retrieval function
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)

# implement the retrieval function
def retrieve(query, top_n=3):
  query_embedding = embed_model.get_text_embedding(query)
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]


In [ ]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')

context = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{context}
'''



Ask me a question: Tomorrow’s weather?
Retrieved knowledge:
 - (similarity: 0.78) It seemed a good initial guess. i would follow the instructions and if the problem persists, I will go to a doctor.It was a good first guess for the treatment. The environment was like a doctor's office, so it felt good. It seemed like a google search for the choice he provided for the condition that I had. I liked the interaction between myself and the doctor.Yes, because it provides a first good guess and I would probably follow it.
 - (similarity: 0.78) Likely to accept but depends on the amount of information I am given and how it matches my own knowledgeCorrect, factual but lacking in detail, would have liked the chance to hear more detailed information or be presented with several treatment choices rather than just oneWould prefer real doctor but would consider AI doctor if faster/cheaper/more convenient
 - (similarity: 0.77) actually I have no reason to denyThe information, knowledge and profession